# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
# 🧪 Install required packages (quiet mode)
!pip install -q requests matplotlib pandas pyinputplus ipywidgets

In [ ]:
import os
# The API part is already integrated into the other segments of my code.

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
# --- IMPORTS ---
import requests
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import re
import pyinputplus as pyip
from typing import Dict, Any
import ipywidgets as widgets
from IPython.display import display, clear_output

## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here

# --- GEOLOCATION FUNCTION ---
def geocode_location(location: str) -> tuple[float, float]:
    # Converts a location name to latitude and longitude using Nominatim API.
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': location, 'format': 'json', 'limit': 1}
    headers = {'User-Agent': 'weather-colab'}  # Required by Nominatim API
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()  # Raise error if request fails
    data = response.json()
    if not data:
        raise ValueError(f"❌ Location '{location}' not found.")
    # Return the first result's latitude and longitude as floats
    return float(data[0]['lat']), float(data[0]['lon'])

# --- FETCH WEATHER DATA FUNCTION ---
def fetch_weather(lat: float, lon: float, start_date: str, end_date: str) -> Dict[str, Any]:
    # Fetches weather data from Open-Meteo API for the given coordinates and date range.
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': lat,
        'longitude': lon,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'current_weather': True,                                                 # Include current weather conditions
        'timezone': 'auto',                                                      # Automatically adjust timezone based
        'start_date': start_date,
        'end_date': end_date
    }
    return requests.get(url, params=params).json()                   # Send GET request to the API with parameters and return the JSON response as a dict

# --- FORECAST DATA EXTRACTION FUNCTION ---
def _extract_forecast_data(weather_data, keys):
    # Utility: Extracts lists of values for specified keys from weather_data['forecast'].
    forecast = weather_data.get('forecast', [])
    return [[day.get(k, None) for day in forecast] for k in keys]    # For each key, build a list of its values from each day in the forecast

# --- MAIN WEATHER DATA FUNCTION ---
def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    # Gets weather data for a location for up to 5 days.
    forecast_days = min(max(forecast_days, 1), 5)                    # Clamp days between 1 and 5
    lat, lon = geocode_location(location)                            # Convert location name to coordinates
    today = datetime.date.today()
    end_date = today + datetime.timedelta(days=forecast_days - 1)
    raw_data = fetch_weather(lat, lon, today.isoformat(), end_date.isoformat())
    # Structure the result
    result = {
        'location': location,
        'latitude': lat,
        'longitude': lon,
        'current': raw_data.get('current_weather', {}),             # Current weather conditions
        'forecast': []
    }
    daily = raw_data.get('daily', {})                               # Iterate through daily forecast data and build a list of daily weather dicts
    # Build the forecast list: each entry is a dict for a day's weather
    for date, t_max, t_min, precip in zip(
        daily.get('time', []),
        daily.get('temperature_2m_max', []),
        daily.get('temperature_2m_min', []),
        daily.get('precipitation_sum', [])
    ):
        result['forecast'].append({
            'date': date,
            'temp_max': t_max,
            'temp_min': t_min,
            'precipitation': precip
        })
    return result

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here

# --- TEMPERATURE VISUALIZATION FUNCTION ---
def create_temperature_visualisation(weather_data, output_type='display'):
    # Plots max and min temperatures as a line chart.
    dates, temp_max, temp_min = _extract_forecast_data(weather_data, ['date', 'temp_max', 'temp_min'])     # Extract dates, max temps, and min temps from the weather data
    fig, ax = plt.subplots(figsize=(10, 5))                                                                # Create a matplotlib figure and axis
    ax.plot(dates, temp_max, label='Max Temp (°C)', marker='o', color='crimson')                           # Plot max temperature as a red line with circle markers
    ax.plot(dates, temp_min, label='Min Temp (°C)', marker='o', color='royalblue')
    ax.fill_between(dates, temp_min, temp_max, color='lightgray', alpha=0.3)               # Fill the area between min and max temperature lines with light gray shading
    ax.set_title(f"Temperature Forecast for {weather_data.get('location', 'Unknown')}")    # Set the plot title with the location name
    ax.set_xlabel("Date")                                                                  # Label the x-axis and y-axis
    ax.set_ylabel("Temperature (°C)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()                                                                      # Adjust layout to prevent clipping of labels
    if output_type == 'figure':                                                             # Return figure or display plot based on output_type
        return fig
    else:
        plt.show()

In [ ]:
# --- PRECIPITATION VISUALIZATION FUNCTION ---
def create_precipitation_visualisation(weather_data, output_type='display'):
    # Plots precipitation as a bar chart.
    forecast = weather_data.get('forecast', [])
    if forecast:
        # Extract dates and precipitation, defaulting missing precipitation to 0
        dates, precipitation = zip(*[
            (day.get('date', 'Unknown'), day.get('precipitation') if day.get('precipitation') is not None else 0)
            for day in forecast if 'date' in day
        ])
    else:
        dates, precipitation = ([], [])
    fig, ax = plt.subplots(figsize=(10, 5))                               # Create a new figure and axis object for the plot with a size of 10x5 inches

    # Plot the precipitation data as a bar chart
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)          # - Bars colored in 'deepskyblue' for visual appeal
    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")                                                 # Label the x-axis as 'Date'
    ax.set_ylabel("Precipitation (mm)")                                   # Label the y-axis as 'Precipitation (mm)'
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()
    if output_type == 'figure':
        return fig
    else:
        plt.show()

## 🤖 Natural Language Processing

In [ ]:
# --- NATURAL LANGUAGE QUESTION PARSING ---
def parse_weather_question(question):
    # Parses a user's question to extract location, time period, and weather attribute.
    original = question                                 # Keep original question for case-sensitive operations later
    question = question.lower()                         # Convert to lowercase for case-insensitive matching
    location = None                                     # Location not yet identified
    time_period = 'today'
    weather_attribute = 'general'
    is_vague = False                                     # Flag to mark if question lacks clarity

    # Detect time period from keywords
    if 'tomorrow' in question:
        time_period = 'tomorrow'                         # User explicitly asked about tomorrow
    elif 'soon' in question or 'later' in question:
        time_period = 'next_1_days'                      # Vague future references interpreted as next 1 day
    else:
        match = re.search(r'next (\d+) days', question)  # Look for pattern like "next 3 days" to extract specific day count
        if match:
            time_period = f"next_{match.group(1)}_days"

    # Detect weather attribute from keywords
    if any(term in question for term in ['temperature', 'temp', 'hot', 'cold']):       # Check if the question contains any terms related to temperature
        weather_attribute = 'temperature'
    elif any(term in question for term in ['rain', 'precipitation', 'snow', 'wet']):   # Check if the question contains any terms related to precipitation or rain
        weather_attribute = 'precipitation'       # Set attribute to precipitation
    elif 'forecast' in question:                  # Check if the question mentions a general forecast
        weather_attribute = 'forecast'            # Set attribute to general forecast
    elif 'humidity' in question:
        weather_attribute = 'humidity'
    elif 'wind' in question:
        weather_attribute = 'wind'

    # Try to extract location: first look for "in <location>", else capitalized words
    location_match = re.search(r'\bin ([a-z\s\-]+)', question)
    if location_match:                                         # If the pattern is found, extract the location string and remove any leading/trailing whitespace
        location = location_match.group(1).strip()
    else:
        # fallback: find capitalized words as location
        caps = re.findall(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)?', original)
        if caps:
            location = ' '.join(caps)                         # Join multiple capitalized words to form multi-word location names

    # Mark as vague if missing key info
    if not location or weather_attribute == 'general':
        is_vague = True

    return {                                              # Return a dictionary with the extracted location, time period, weather attribute, and vagueness flag
        'location': location,
        'time_period': time_period,
        'weather_attribute': weather_attribute,
        'is_vague': is_vague
    }

# --- GENERATE WEATHER RESPONSE ---
def generate_weather_response(parsed: dict, weather_data: dict) -> str:
    # Returns a human-readable response based on the parsed question and weather data.
    location = parsed['location']
    time = parsed['time_period']
    attr = parsed['weather_attribute']
    forecast = weather_data.get('forecast', [])                                  # List of daily forecasts
    current = weather_data.get('current', {})                                    # Current weather data

    if parsed['is_vague']:                                                       # If the question is vague (missing location or attribute), prompt user for more info
        return "❗ Please include a location and weather detail (e.g. temperature or rain)."

    # Helper: find forecast for a specific date
    def forecast_for_date(date_str):
        for day in forecast:
            if day['date'] == date_str:
                return day
        return None

    today = datetime.date.today()                                            # Determine the target date based on the requested time period
    if time == 'today':
        target_date = today.isoformat()
    elif time == 'tomorrow':
        target_date = (today + datetime.timedelta(days=1)).isoformat()
    elif time.startswith('next_'):
        # For multi-day forecasts, just return summary
        days = int(time.split('_')[1])                                      # For multi-day forecasts, extract the number of days requested
        temps = [f"{d['date']}: {d['temp_min']}°C - {d['temp_max']}°C" for d in forecast[:days]]
        return "🌡️ Temperature forecast:\n" + "\n".join(temps)             # Return the multi-day temperature forecast as a formatted string
    else:
        target_date = today.isoformat()

    day_forecast = forecast_for_date(target_date)                          # Retrieve the forecast for the target date
    if not day_forecast:
        return f"⚠️ No forecast data available for {target_date}."

    # Handle precipitation (rain/snow)
    if attr == 'precipitation':                                            # Precipitation (rain, snow, wetness)
        precip = day_forecast.get('precipitation', 0)
        if precip > 0:
            return f"🌧️ Yes, it looks like there will be precipitation ({precip} mm) in {location} on {target_date}."
        else:
            return f"❌ No, it doesn't look like it will rain or snow in {location} on {target_date}."

    # Handle temperature
    elif attr == 'temperature':                                             # Temperature range
        return (f"🌡️ On {target_date} in {location}, the temperature will range from "
                f"{day_forecast['temp_min']}°C to {day_forecast['temp_max']}°C.")
    # Handle wind
    elif attr == 'wind':
        windspeed = current.get('windspeed', 'unknown')                     # Wind speed (from current weather data)
        return f"💨 Wind speed in {location} is {windspeed} km/h."
    # Handle humidity (not available)
    elif attr == 'humidity':                                                # Humidity (not available in this API)
        return "💧 Sorry, humidity data is not available from this API."

    # Generic fallback
    return "⚠️ Could not understand your weather request."  # Fallback response if attribute is unrecognized

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

# --- IPYWIDGETS UI INTEGRATION ---

# Widgets for user input
location_input = widgets.Text(
    description='📍 Location:',                            # Label shown next to the input box
    placeholder='e.g., Sydney',                            # Placeholder text inside the input box
    style={'description_width': 'initial'}                 # Style to control label width for better layout
)

# Text input widget for the user to enter a natural language weather question
question_input = widgets.Text(
    description='💬 Ask:',
    placeholder='e.g., Will it rain in Berlin tomorrow?',    # Placeholder text
    style={'description_width': 'initial'}                   # Style for label width
)

# Custom colored buttons
ask_button = widgets.Button(
    description='Ask Weather',                 # Button text
    button_style='',                           # No predefined style
    layout=widgets.Layout(width='150px')
)
ask_button.style.button_color = '#3498db'      # Blue

temp_button = widgets.Button(
    description='Show Temperature',
    button_style='success',  # Green
    layout=widgets.Layout(width='150px')      # Fixed width for consistent UI
)
# No custom color needed; 'success' is green

precip_button = widgets.Button(
    description='Show Precipitation',         # Button text
    button_style='',                          # No predefined style
    layout=widgets.Layout(width='180px')
)
precip_button.style.button_color = '#f39c12'  # Orange

output_area = widgets.Output()

# --- HANDLERS FOR BUTTONS ---
def on_ask_clicked(b):
    # Handles when the user clicks "Ask Weather"
    with output_area:
        clear_output()                                     # Clear previous output to keep display clean
        question = question_input.value.strip()            # Get the question text, trimmed of whitespace
        parsed = parse_weather_question(question)
        if not parsed['location']:
            print("❌ Please specify a location.")         # If no location was found in the question, prompt the user to specify one
            return
        try:
            data = get_weather_data(parsed['location'])
            print(generate_weather_response(parsed, data)) # Generate and print a user-friendly weather response based on parsed question and data
        except Exception as e:                             # If an error occurs (e.g., invalid location, network issue), print an error message
            print(f"⚠️ {e}")

def on_temp_clicked(b):
    # Handles when the user clicks "Show Temperature"
    with output_area:
        clear_output()
        loc = location_input.value.strip()     # Get the location input from the user, removing extra spaces
        if not loc:
            print("❌ Enter a location.")     # If no location is entered, show an error message and exit the function
            return
        try:
            data = get_weather_data(loc)
            create_temperature_visualisation(data)     # Create and display the temperature visualization using the fetched data
        except Exception as e:
            print(f"⚠️ {e}")                           # If any error occurs (e.g., invalid location or network issue), display an error message

def on_precip_clicked(b):
    # Handles when the user clicks "Show Precipitation"
    with output_area:
        clear_output()                        # Clear previous output to keep the display clean
        loc = location_input.value.strip()    # Get the location input from the user, trimming whitespace
        if not loc:
            print("❌ Enter a location.")     # If no location is entered, show an error message and exit the function
            return
        try:
            data = get_weather_data(loc)      # Attempt to fetch weather data for the entered location
            create_precipitation_visualisation(data)
        except Exception as e:                # If any error occurs (e.g., invalid location or network issue), display an error message
            print(f"⚠️ {e}")

# Link buttons to their handlers
ask_button.on_click(on_ask_clicked)           # Connect "Ask Weather" button to its handler
temp_button.on_click(on_temp_clicked)         # Connect "Show Temperature" button to its handler
precip_button.on_click(on_precip_clicked)     # Connect "Show Precipitation" button to its handler

# --- LAYOUT AND DISPLAY ---
ui = widgets.VBox([
    widgets.HTML("<h2>☁️ Weather Assistant</h2>"),    # HTML widget to display a heading for the app with a cloud emoji
    question_input,                                   # Text input widget where users can type their natural language weather question
    ask_button,                                       # Button widget for submitting the weather question ("Ask Weather")
    widgets.HTML("<hr>"),
    location_input,
    widgets.HBox([temp_button, precip_button]),      # Horizontal box (HBox) layout to place the temperature and precipitation buttons side by side
    output_area                                      # Output widget area to display results, messages, or plots generated by the app
])

# Display the interactive UI in the notebook
display(ui)

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here

# 🧪 Install required packages (quiet mode)
!pip install -q requests matplotlib pandas pyinputplus ipywidgets

# ✅ Imports
import requests
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import re
import pyinputplus as pyip
from typing import Dict, Any
import ipywidgets as widgets
from IPython.display import display, clear_output

# ✅ Geocoding using OpenStreetMap Nominatim
def geocode_location(location: str) -> tuple[float, float]:
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': location, 'format': 'json', 'limit': 1}
    headers = {'User-Agent': 'weather-colab'}  # Required by Nominatim
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    if not data:
        raise ValueError(f"❌ Location '{location}' not found.")
    return float(data[0]['lat']), float(data[0]['lon'])

# ✅ Fetch weather data from Open-Meteo API
def fetch_weather(lat: float, lon: float, start_date: str, end_date: str) -> Dict[str, Any]:
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': lat,
        'longitude': lon,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'current_weather': True,
        'timezone': 'auto',
        'start_date': start_date,
        'end_date': end_date
    }
    return requests.get(url, params=params).json()

# ✅ Extract forecast data
def _extract_forecast_data(weather_data, keys):
    forecast = weather_data.get('forecast', [])
    return [[day.get(k, None) for day in forecast] for k in keys]

# ✅ Get full weather data
def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    forecast_days = min(max(forecast_days, 1), 5)
    lat, lon = geocode_location(location)
    today = datetime.date.today()
    end_date = today + datetime.timedelta(days=forecast_days - 1)
    raw_data = fetch_weather(lat, lon, today.isoformat(), end_date.isoformat())
    result = {
        'location': location,
        'latitude': lat,
        'longitude': lon,
        'current': raw_data.get('current_weather', {}),
        'forecast': []
    }
    daily = raw_data.get('daily', {})
    for date, t_max, t_min, precip in zip(
        daily.get('time', []),
        daily.get('temperature_2m_max', []),
        daily.get('temperature_2m_min', []),
        daily.get('precipitation_sum', [])
    ):
        result['forecast'].append({
            'date': date,
            'temp_max': t_max,
            'temp_min': t_min,
            'precipitation': precip
        })
    return result

# ✅ Plot temperature
def create_temperature_visualisation(weather_data, output_type='display'):
    dates, temp_max, temp_min = _extract_forecast_data(weather_data, ['date', 'temp_max', 'temp_min'])
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(dates, temp_max, label='Max Temp (°C)', marker='o', color='crimson')
    ax.plot(dates, temp_min, label='Min Temp (°C)', marker='o', color='royalblue')
    ax.fill_between(dates, temp_min, temp_max, color='lightgray', alpha=0.3)
    ax.set_title(f"Temperature Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Temperature (°C)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()
    if output_type == 'figure':
        return fig
    else:
        plt.show()

# ✅ Plot precipitation
def create_precipitation_visualisation(weather_data, output_type='display'):
    forecast = weather_data.get('forecast', [])
    if forecast:
        dates, precipitation = zip(*[
            (day.get('date', 'Unknown'), day.get('precipitation') if day.get('precipitation') is not None else 0)
            for day in forecast if 'date' in day
        ])
    else:
        dates, precipitation = ([], [])
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)
    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")
    ax.set_ylabel("Precipitation (mm)")
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()
    if output_type == 'figure':
        return fig
    else:
        plt.show()

# ✅ Natural language parsing
def parse_weather_question(question):
    original = question
    question = question.lower()
    location = None
    time_period = 'today'
    weather_attribute = 'general'
    is_vague = False

    if 'tomorrow' in question:
        time_period = 'tomorrow'
    elif 'soon' in question or 'later' in question:
        time_period = 'next_1_days'
    else:
        match = re.search(r'next (\d+) days', question)
        if match:
            time_period = f"next_{match.group(1)}_days"

    if any(term in question for term in ['temperature', 'temp', 'hot', 'cold']):
        weather_attribute = 'temperature'
    elif any(term in question for term in ['rain', 'precipitation', 'snow', 'wet']):
        weather_attribute = 'precipitation'
    elif 'forecast' in question:
        weather_attribute = 'forecast'
    elif 'humidity' in question:
        weather_attribute = 'humidity'
    elif 'wind' in question:
        weather_attribute = 'wind'

    location_match = re.search(r'\bin ([a-z\s\-]+)', question)
    if location_match:
        location = location_match.group(1).strip()
    else:
        # fallback: find capitalized words as location
        caps = re.findall(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)?', original)
        if caps:
            location = ' '.join(caps)

    if not location or weather_attribute == 'general':
        is_vague = True

    return {
        'location': location,
        'time_period': time_period,
        'weather_attribute': weather_attribute,
        'is_vague': is_vague
    }

# ✅ Generate a response
def generate_weather_response(parsed: dict, weather_data: dict) -> str:
    location = parsed['location']
    time = parsed['time_period']
    attr = parsed['weather_attribute']
    forecast = weather_data.get('forecast', [])
    current = weather_data.get('current', {})

    if parsed['is_vague']:
        return "❗ Please include a location and weather detail (e.g. temperature or rain)."

    # Helper: find forecast for a specific date
    def forecast_for_date(date_str):
        for day in forecast:
            if day['date'] == date_str:
                return day
        return None

    today = datetime.date.today()
    if time == 'today':
        target_date = today.isoformat()
    elif time == 'tomorrow':
        target_date = (today + datetime.timedelta(days=1)).isoformat()
    elif time.startswith('next_'):
        # For multi-day forecasts, just return summary
        days = int(time.split('_')[1])
        temps = [f"{d['date']}: {d['temp_min']}°C - {d['temp_max']}°C" for d in forecast[:days]]
        return "🌡️ Temperature forecast:\n" + "\n".join(temps)
    else:
        target_date = today.isoformat()

    day_forecast = forecast_for_date(target_date)
    if not day_forecast:
        return f"⚠️ No forecast data available for {target_date}."

    # Handle precipitation (rain/snow)
    if attr == 'precipitation':
        precip = day_forecast.get('precipitation', 0)
        if precip > 0:
            return f"🌧️ Yes, it looks like there will be precipitation ({precip} mm) in {location} on {target_date}."
        else:
            return f"❌ No, it doesn't look like it will rain or snow in {location} on {target_date}."

    # Handle temperature
    elif attr == 'temperature':
        return (f"🌡️ On {target_date} in {location}, the temperature will range from "
                f"{day_forecast['temp_min']}°C to {day_forecast['temp_max']}°C.")

    # Handle wind
    elif attr == 'wind':
        windspeed = current.get('windspeed', 'unknown')
        return f"💨 Wind speed in {location} is {windspeed} km/h."
    # Handle humidity (not available)
    elif attr == 'humidity':
        return "💧 Sorry, humidity data is not available from this API."

    # Generic fallback
    return "⚠️ Could not understand your weather request."

# ====== IPyWidgets UI Integration ======

# Widgets
location_input = widgets.Text(
    description='📍 Location:',
    placeholder='e.g., Sydney',
    style={'description_width': 'initial'}
)

question_input = widgets.Text(
    description='💬 Ask:',
    placeholder='e.g., Will it rain in Berlin tomorrow?',
    style={'description_width': 'initial'}
)

# Custom colored buttons
ask_button = widgets.Button(
    description='Ask Weather',
    button_style='',  # No predefined style
    layout=widgets.Layout(width='150px')
)
ask_button.style.button_color = '#3498db'  # Blue

temp_button = widgets.Button(
    description='Show Temperature',
    button_style='success',  # Green
    layout=widgets.Layout(width='150px')
)
# No custom color needed; 'success' is green

precip_button = widgets.Button(
    description='Show Precipitation',
    button_style='',  # No predefined style
    layout=widgets.Layout(width='180px')
)
precip_button.style.button_color = '#f39c12'  # Orange

output_area = widgets.Output()

# Handlers
def on_ask_clicked(b):
    with output_area:
        clear_output()
        question = question_input.value.strip()
        parsed = parse_weather_question(question)
        if not parsed['location']:
            print("❌ Please specify a location.")
            return
        try:
            data = get_weather_data(parsed['location'])
            print(generate_weather_response(parsed, data))
        except Exception as e:
            print(f"⚠️ {e}")

def on_temp_clicked(b):
    with output_area:
        clear_output()
        loc = location_input.value.strip()
        if not loc:
            print("❌ Enter a location.")
            return
        try:
            data = get_weather_data(loc)
            create_temperature_visualisation(data)
        except Exception as e:
            print(f"⚠️ {e}")

def on_precip_clicked(b):
    with output_area:
        clear_output()
        loc = location_input.value.strip()
        if not loc:
            print("❌ Enter a location.")
            return
        try:
            data = get_weather_data(loc)
            create_precipitation_visualisation(data)
        except Exception as e:
            print(f"⚠️ {e}")

# Link buttons
ask_button.on_click(on_ask_clicked)
temp_button.on_click(on_temp_clicked)
precip_button.on_click(on_precip_clicked)

# Layout
ui = widgets.VBox([
    widgets.HTML("<h2>☁️ Weather Assistant</h2>"),
    question_input,
    ask_button,
    widgets.HTML("<hr>"),
    location_input,
    widgets.HBox([temp_button, precip_button]),
    output_area
])

# Display the UI
display(ui)

## 🧪 Testing and Examples

In [ ]:
#1. 📘 geocode_location(location: str) -> tuple[float, float]
def geocode_location(location: str) -> tuple[float, float]:
    """
    Geocode a location using OpenStreetMap Nominatim API.

    Args:
        location (str): The name of the location to geocode.

    Returns:
        tuple: A tuple containing the latitude and longitude.

    >>> isinstance(geocode_location("Paris"), tuple)
    True
    """
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': location, 'format': 'json', 'limit': 1}
    headers = {'User-Agent': 'weather-colab'}
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()
    if not data:
        raise ValueError(f"❌ Location '{location}' not found.")
    return float(data[0]['lat']), float(data[0]['lon'])
#unittest + assert + 🐞 debug():

import unittest

class TestGeoCode(unittest.TestCase):
    def test_known_location(self):
        lat, lon = geocode_location("Berlin")
        self.assertTrue(-90 <= lat <= 90 and -180 <= lon <= 180)

# ✅ Run unit test
unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestGeoCode))

# ✅ Assertion Test
lat, lon = geocode_location("Tokyo")
assert isinstance(lat, float) and isinstance(lon, float)

# 🐞 Debug Simulation
def debug_geocode():
    try:
        print("Debug: Geocoding 'New York'")
        print(geocode_location("New York"))
    except Exception as e:
        print("❌", e)

debug_geocode()

#2. 📘 fetch_weather(lat: float, lon: float, start_date: str, end_date: str)
def fetch_weather(lat: float, lon: float, start_date: str, end_date: str) -> Dict[str, Any]:
    """
    Fetches weather data from the Open-Meteo API.

    Args:
        lat (float): Latitude.
        lon (float): Longitude.
        start_date (str): ISO date (yyyy-mm-dd).
        end_date (str): ISO date (yyyy-mm-dd).

    Returns:
        dict: JSON response with weather data.
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': lat,
        'longitude': lon,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'current_weather': True,
        'timezone': 'auto',
        'start_date': start_date,
        'end_date': end_date
    }
    return requests.get(url, params=params).json()
#✅ unittest + assert + 🐞 debug():

class TestFetchWeather(unittest.TestCase):
    def test_structure(self):
        today = datetime.date.today()
        data = fetch_weather(48.8566, 2.3522, today.isoformat(), today.isoformat())
        self.assertIn("current_weather", data)

unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestFetchWeather))

# ✅ Assertion
today = datetime.date.today().isoformat()
assert "current_weather" in fetch_weather(51.5074, -0.1278, today, today)

# 🐞 Debug
def debug_fetch_weather():
    today = datetime.date.today().isoformat()
    print(fetch_weather(40.7128, -74.0060, today, today))

debug_fetch_weather()
3. 📘 _extract_forecast_data(weather_data, keys)
def _extract_forecast_data(weather_data, keys):
    """
    Extract specific keys from forecast.

    Args:
        weather_data (dict): Weather data.
        keys (list): Keys to extract from forecast.

    Returns:
        list: Lists of values for each key.
    """
    forecast = weather_data.get('forecast', [])
    return [[day.get(k, None) for day in forecast] for k in keys]
✅ doctest:

def test_extract_forecast_data():
    """
    >>> data = {'forecast': [{'date': '2024-01-01', 'temp_max': 30}]}
    >>> _extract_forecast_data(data, ['date', 'temp_max'])
    [['2024-01-01'], [30]]
    """
    pass
#✅ unittest + assert + 🐞 debug():

class TestExtractForecast(unittest.TestCase):
    def test_extract(self):
        dummy = {'forecast': [{'date': '2024-01-01', 'temp_max': 25, 'temp_min': 15}]}
        result = _extract_forecast_data(dummy, ['date', 'temp_max'])
        self.assertEqual(result, [['2024-01-01'], [25]])

unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestExtractForecast))

# ✅ Assertion
dummy = {'forecast': [{'date': '2024-01-01', 'temp_max': 28}]}
assert _extract_forecast_data(dummy, ['date']) == [['2024-01-01']]

# 🐞 Debug
def debug_extract():
    test = {'forecast': [{'date': '2025-01-01', 'temp_max': 32, 'temp_min': 18}]}
    print(_extract_forecast_data(test, ['date', 'temp_max', 'temp_min']))

debug_extract()
Great! Let’s continue with the next set of functions, using the same enhanced structure:

#4. 📘 get_weather_data(location: str, days: int = 7) -> Dict[str, Any]
def get_weather_data(location: str, days: int = 7) -> Dict[str, Any]:
    """
    Fetch and organize weather data for a given location.

    Args:
        location (str): Location to retrieve weather data for.
        days (int): Number of days of forecast (default: 7).

    Returns:
        dict: Contains location, current weather, and daily forecast.

    >>> data = get_weather_data("Rome", 1)
    >>> "location" in data and "current" in data
    True
    """
    lat, lon = geocode_location(location)
    today = datetime.date.today()
    end = today + datetime.timedelta(days=days)
    raw = fetch_weather(lat, lon, today.isoformat(), end.isoformat())
    return {
        "location": location,
        "current": raw.get("current_weather", {}),
        "forecast": [
            {
                "date": date,
                "temp_max": tmax,
                "temp_min": tmin,
                "precipitation": rain
            }
            for date, tmax, tmin, rain in zip(
                raw["daily"]["time"],
                raw["daily"]["temperature_2m_max"],
                raw["daily"]["temperature_2m_min"],
                raw["daily"]["precipitation_sum"]
            )
        ]
    }
#✅ unittest + assert + 🐞 debug():

class TestWeatherData(unittest.TestCase):
    def test_structure_keys(self):
        data = get_weather_data("Madrid", 1)
        self.assertIn("location", data)
        self.assertIn("current", data)
        self.assertTrue(len(data["forecast"]) >= 1)

unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestWeatherData))

# ✅ Assertion
data = get_weather_data("Lisbon", 1)
assert "forecast" in data and isinstance(data["forecast"], list)

# 🐞 Debug
def debug_weather_data():
    print("Debug: Getting weather data for Sydney")
    result = get_weather_data("Sydney", 2)
    print(result)

debug_weather_data()
5. 📘 create_temperature_visualisation(data: Dict[str, Any]) -> go.Figure
def create_temperature_visualisation(data: Dict[str, Any]) -> go.Figure:
    """
    Create a line chart for temperature forecast.

    Args:
        data (dict): Weather data with daily forecast.

    Returns:
        plotly.graph_objects.Figure: Line chart figure.
    """
    forecast = data.get("forecast", [])
    dates = [d["date"] for d in forecast]
    temp_max = [d["temp_max"] for d in forecast]
    temp_min = [d["temp_min"] for d in forecast]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=temp_max, mode='lines+markers', name='Max Temp'))
    fig.add_trace(go.Scatter(x=dates, y=temp_min, mode='lines+markers', name='Min Temp'))
    fig.update_layout(
        title=f"Temperature Forecast for {data.get('location', '')}",
        xaxis_title="Date",
        yaxis_title="Temperature (°C)"
    )
    return fig
#✅ unittest + assert + 🐞 debug():

class TestTemperatureViz(unittest.TestCase):
    def test_fig_output(self):
        mock_data = {
            "location": "Testville",
            "forecast": [
                {"date": "2024-01-01", "temp_max": 25, "temp_min": 15},
                {"date": "2024-01-02", "temp_max": 26, "temp_min": 14}
            ]
        }
        fig = create_temperature_visualisation(mock_data)
        self.assertIsInstance(fig, go.Figure)

unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestTemperatureViz))

# ✅ Assertion
mock = {
    "location": "FakeCity",
    "forecast": [{"date": "2024-01-01", "temp_max": 20, "temp_min": 10}]
}
fig = create_temperature_visualisation(mock)
assert isinstance(fig, go.Figure)

# 🐞 Debug
def debug_temp_viz():
    mock = {
        "location": "DebugCity",
        "forecast": [{"date": "2024-01-01", "temp_max": 30, "temp_min": 20}]
    }
    fig = create_temperature_visualisation(mock)
    fig.show()

debug_temp_viz()
#6. 📘 create_precipitation_visualisation(data: Dict[str, Any]) -> go.Figure
def create_precipitation_visualisation(data: Dict[str, Any]) -> go.Figure:
    """
    Create a bar chart for precipitation forecast.

    Args:
        data (dict): Weather data with daily forecast.

    Returns:
        plotly.graph_objects.Figure: Bar chart figure.
    """
    forecast = data.get("forecast", [])
    dates = [d["date"] for d in forecast]
    rain = [d["precipitation"] for d in forecast]

    fig = go.Figure()
    fig.add_trace(go.Bar(x=dates, y=rain, name='Precipitation'))
    fig.update_layout(
        title=f"Precipitation Forecast for {data.get('location', '')}",
        xaxis_title="Date",
        yaxis_title="Precipitation (mm)"
    )
    return fig
#✅ unittest + assert + 🐞 debug():

class TestPrecipitationViz(unittest.TestCase):
    def test_fig_output(self):
        mock = {
            "location": "Rainville",
            "forecast": [
                {"date": "2024-01-01", "precipitation": 5},
                {"date": "2024-01-02", "precipitation": 10}
            ]
        }
        fig = create_precipitation_visualisation(mock)
        self.assertIsInstance(fig, go.Figure)

unittest.TextTestRunner().run(unittest.TestLoader().loadTestsFromTestCase(TestPrecipitationViz))

# ✅ Assertion
mock = {
    "location": "Drytown",
    "forecast": [{"date": "2024-01-01", "precipitation": 0}]
}
assert isinstance(create_precipitation_visualisation(mock), go.Figure)

# 🐞 Debug
def debug_precip_viz():
    mock = {
        "location": "StormCity",
        "forecast": [{"date": "2024-01-01", "precipitation": 15}]
    }
    fig = create_precipitation_visualisation(mock)
    fig.show()

debug_precip_viz()

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.